#Read data

In [1]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
words_revert={words[i][1]:i for i in words}#feminine:masculine
#neutral=load_simple_json('de/neutral_new.json')

no_page_m=load_simple_json('de/wiki/m_no_page.json')
no_page_f=load_simple_json('de/wiki/f_no_page.json')
#no_page_n=load_simple_json('de/wiki/n_no_page.json')
page_m=load_simple_json('de/wiki/m_page.json')
page_f=load_simple_json('de/wiki/f_page.json')
#page_n=load_simple_json('de/wiki/n_page.json')
m_page_levenshtein=load_simple_json('de/wiki/m_page_levenshtein.json')
#n_page_levenshtein=load_simple_json('de/wiki/n_page_levenshtein.json')


page_f_validated=load_simple_json('de/wiki/f_page_validated.json')
page_f_ambigious=load_simple_json('de/wiki/f_page_ambigious.json')
page_f_other=load_simple_json('de/wiki/f_page_other.json')  
    
page_m_validated=load_simple_json('de/wiki/m_page_validated.json') 
page_m_ambigious=load_simple_json('de/wiki/m_page_ambigious.json')
page_m_other=load_simple_json('de/wiki/m_page_other.json')

#page_n_validated=load_simple_json('de/wiki/n_page_validated.json')
#page_n_ambigious=load_simple_json('de/wiki/n_page_ambigious.json')
#page_n_other=load_simple_json('de/wiki/n_page_other.json') 

m_links_to_feminine=load_simple_json('de/wiki/m_links_to_feminine.json')
m_links_to_smth=load_simple_json('de/wiki/m_links_to_smth.json')
f_links_to_mascuilne=load_simple_json('de/wiki/f_links_to_mascuilne.json')
f_links_to_smth=load_simple_json('de/wiki/f_links_to_smth.json')
#redirection_n=load_simple_json('de/wiki/n_redirects.json')
f_links_to_mascuilne_lev=load_simple_json('de/wiki/f_links_to_mascuilne_lev.json')



with open("de/googlenumber_new2.json", 'r') as f:
    googlenumber=json.load(f)

#Prediction function

In [24]:
#prediction function
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
def predict_linear_regr(df,C_val=1e5,multi_class='ovr',n=1):#'multinomial'
    X = df.get_values()[:,:n] # we only take the first two features.
    Y = df.Wiki
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    print "train and test data size:",X_train.shape, X_test.shape
    logreg = linear_model.LogisticRegression(C=C_val,solver='newton-cg',multi_class='multinomial')# lbfgs’ & ‘newton-cg’ solvers
    # we create an instance of Neighbours Classifier and fit the data.
    logreg_fit=logreg.fit(X_train, y_train)
    #print logreg_fit.intercept_
    y_pred = logreg_fit.predict(X_test)
    print"{0} / {1} correct ({2})".format(np.sum(y_test == y_pred), len(y_test),np.sum(y_test == y_pred)/float(len(y_test)))
    print "Last 90 predicted values:",y_pred[-90:]
    return y_pred

# simple Google hits, predict if wiki page exist

In [22]:
#dependent var
#(i) has WP article entry with text (real article), #1
#(ii) has entry that is a redirect to other gender profession name, #2
#(iii) has entry that is a redirect to something else, #3
#(iv) has no entry at all #4
#use as independent vars /predictors:
#(i) total number of google hits, 
#(ii) relative hits in relation to gender counterpart (i would suggest to just sum up hits for male and female and take the percentage for the target gender), 
#(iii) chars in profession name (just for control)


#male
google_rank_male={}
for i in words:
    if (i in page_m_validated):
        google_rank_male[i]=[int(googlenumber[i][0]),1]
    elif (i in m_links_to_feminine):
        google_rank_male[i]=[int(googlenumber[i][0]),2]
    elif (i in m_links_to_smth) :#3
        google_rank_male[i]=[int(googlenumber[i][0]),3]
    #elif (i in no_page_m):
    else:
        google_rank_male[i]=[int(googlenumber[i][0]),4]

        
        
len(google_rank_male)
import pandas as pd
df = pd.DataFrame.from_dict(google_rank_male,orient='index')
df.columns = ['GoogleResults', 'Wiki']
res1=predict_linear_regr(df)

for i in res1:
    if i!=4:
        print 'Not equal "4": ',i

train and test data size: (3123, 1) (1041, 1)
733 / 1041 correct (0.704130643612)
Last 90 predicted values: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


#use difference

In [27]:
google_dif={}
for i in googlenumber:
    google_dif[i]=int(googlenumber[i][0])-int(googlenumber[i][1])

#start with male bias
google_rank_male={}
for i in words:
    if (i in page_m):
        google_rank_male[i]=[google_dif[i],1]
    elif (i in m_links_to_feminine):
        google_rank_male[i]=[google_dif[i],2]
    elif (i in m_links_to_smth) :#3
        google_rank_male[i]=[google_dif[i],3]
    elif (i in no_page_m):
        google_rank_male[i]=[google_dif[i],4]

len(google_rank_male)

import pandas as pd
df = pd.DataFrame.from_dict(google_rank_male,orient='index')
df.columns = ['GoogleResults', 'Wiki']
res2=predict_linear_regr(df)
n=0
for i in res2:
    if i!=4:
        n+=1
print 'Not equal  "4" {0} times'.format(n)

train and test data size: (3123, 1) (1041, 1)
724 / 1041 correct (0.695485110471)
Last 90 predicted values: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4]
Not equal  "4" 34 times


#use percentage
##male prediction

In [15]:
google_dif={}
for i in googlenumber:
    sum_m_f=float(int(googlenumber[i][0])+int(googlenumber[i][1]))
    google_dif[i]=100*float(googlenumber[i][0])/sum_m_f if sum_m_f!=0 else 0

#start with male bias
google_rank_male={}
for i in words:
    if (i in page_m):
        google_rank_male[i]=[google_dif[i],1]
    elif (i in m_links_to_feminine):
        google_rank_male[i]=[google_dif[i],2]
    elif (i in m_links_to_smth) :#3
        google_rank_male[i]=[google_dif[i],3]
    elif (i in no_page_m):
        google_rank_male[i]=[google_dif[i],4]

len(google_rank_male)

import pandas as pd
df = pd.DataFrame.from_dict(google_rank_male,orient='index')
df.columns = ['GoogleResults', 'Wiki']
print df


                                   GoogleResults  Wiki
Revisor                                99.346950     1
Krankenversicherungskalkulator         70.000000     4
Aquariumbauer                          99.820144     4
Planungstechniker                      91.769547     4
Ozeanograph                            88.412960     3
Immobilienbetriebswirt                 83.042394     4
Radio- und Fernsehtechniker            98.284128     3
Finanzdienstleister                    99.522780     3
NC-Anwendungsfachmann                  95.106383     4
Bekleidungsteilenäher                  80.555556     4
Buchfertigmacher                       91.428571     4
Rohrvorrichter                         98.870056     3
Familiengesundheitspfleger             65.000000     4
Metallformer                           98.953378     4
Heilerziehungspflegehelfer             83.843618     3
Komparse                               82.323232     1
Direktionsbevollmächtigter             82.352941     4
Erwachsene

In [16]:
res3=predict_linear_regr(df)

for i in res3:
    if i!=4:
        print "Not equal 4: ",i
        
res3=predict_linear_regr(df,C_val=1)

for i in res3:
    if i!=4:
        print "Not equal 4: ",i

train and test data size: (3123, 1) (1041, 1)
704 / 1041 correct (0.676272814601)
Last 90 predicted values: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
train and test data size: (3123, 1) (1041, 1)
682 / 1041 correct (0.655139289145)
Last 90 predicted values: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


##female prediction

In [ ]:

google_dif={}
for i in googlenumber:
    sum_m_f=float(int(googlenumber[i][0])+int(googlenumber[i][1]))
    google_dif[i]=100*float(googlenumber[i][1])/sum_m_f if sum_m_f!=0 else 0

#start with male bias
google_rank_male={}
for i in words:
    if (words[i][1] in page_f):
        google_rank_male[i]=[google_dif[i],1]
    elif (words[i][1] in f_links_to_mascuilne):
        google_rank_male[i]=[google_dif[i],2]
    elif (words[i][1] in f_links_to_smth) :#3
        google_rank_male[i]=[google_dif[i],3]
    elif (words[i][1] in no_page_f):
        google_rank_male[i]=[google_dif[i],4]

len(google_rank_male)

import pandas as pd
df = pd.DataFrame.from_dict(google_rank_male,orient='index')
df.columns = ['GoogleResults', 'Wiki']
print df


In [18]:
res4=predict_linear_regr(df)

for i in res4:
    if i!=4:
        print "Not equal 4: ",i

res4=predict_linear_regr(df,multi_class='multinomial')
for i in res4:
    if i!=4:
        print "Not equal 4: ",i

train and test data size: (3123, 1) (1041, 1)
974 / 1041 correct (0.935638808838)
Last 90 predicted values: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
train and test data size: (3123, 1) (1041, 1)
962 / 1041 correct (0.924111431316)
Last 90 predicted values: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


##all together

In [31]:
google_dif={}
for i in googlenumber:
    google_dif[i]=int(googlenumber[i][0])-int(googlenumber[i][1])

google_percent={}
for i in googlenumber:
    sum_m_f=float(int(googlenumber[i][0])+int(googlenumber[i][1]))
    google_percent[i]=100*float(googlenumber[i][0])/sum_m_f if sum_m_f!=0 else 0
#start with male bias
google_rank_male={}
for i in words:
    if (i in page_m):
        google_rank_male[i]=[int(googlenumber[i][0]),google_dif[i],google_percent[i],1]
    elif (i in m_links_to_feminine):
        google_rank_male[i]=[int(googlenumber[i][0]),google_dif[i],google_percent[i],2]
    elif (i in m_links_to_smth) :#3
        google_rank_male[i]=[int(googlenumber[i][0]),google_dif[i],google_percent[i],3]
    elif (i in no_page_m):
        google_rank_male[i]=[int(googlenumber[i][0]),google_dif[i],google_percent[i],4]

len(google_rank_male)

import pandas as pd
df = pd.DataFrame.from_dict(google_rank_male,orient='index')
df.columns = ['GoogleResults','GoogleDif', 'GooglePercent','Wiki']
res2=predict_linear_regr(df,n=3)
n=0
for i in res2:
    if i!=4:
        n+=1
print 'Not equal  "4" {0} times'.format(n)

train and test data size: (3123, 3) (1041, 3)
734 / 1041 correct (0.705091258405)
Last 90 predicted values: [4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 1 4 4 4 1 4 4 4 4 4
 4 1 4 1 4 4 4 4 4 4 4 4 1 4 4 4 4 4 1 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
Not equal  "4" 73 times


#Trash & old

In [ ]:
#count: masculine exist AND (no feminine OR feminine label redirects to masculine label) => male bias (Rank 1)
#count: feminine exist AND (no masculine OR masculine label redirects to feminine label)  => female bias (Rank 3)
#count: masculine exist AND feminine exist => balanced (Rank 2)

#start with male bias
male_bias_labels=[]
female_bias_labels=[]
no_bias=[]
wiki_bias_rank={}
for i in words:
    if ((i in page_m)&(words[i][1] not in page_f))|((i in page_m)&(i in m_links_to_feminine)):
        male_bias_labels.append(i)
        wiki_bias_rank[i]=1
    elif ((words[i][1] in page_f)&(i not in page_m))|((words[i][1] in page_f)&(i in f_links_to_mascuilne)):
        female_bias_labels.append(i)
        wiki_bias_rank[i]=3
        #print i
    elif (i in page_m)&(words[i][1] in page_f):
        no_bias.append(i)
        wiki_bias_rank[i]=2
        #print i
    else:
        wiki_bias_rank[i]=0
print "Male bias:",len(male_bias_labels),", Female bias:",len(female_bias_labels),", No bias:",len(no_bias)

with open("de/googlenumber_new2.json", 'r') as f:
    googlenumber=json.load(f)
google_rank_male={}
for i in words:
    if wiki_bias_rank.has_key(i):
        google_rank_male[i]=[int(googlenumber[i][0]),wiki_bias_rank[i]]      
        
import pandas as pd
df = pd.DataFrame.from_dict(google_rank_male,orient='index')
df.columns = ['GoogleResults', 'Wiki']
predict_linear_regr(df)

In [ ]:
import json
#google Results
#difference would be our Goosle Runking
with open("de/googlenumber_new.json", 'r') as f:
    googlenumber=json.load(f)
google_dif={}
for i in googlenumber:
    google_dif[i]=int(googlenumber[i][0])-int(googlenumber[i][1])
    
#Rank these values
from scipy import stats
google_rank=stats.rankdata(google_dif.values())
google_rank_labels=google_dif.keys()
google_rank_dict={}
print google_dif.keys().index(u'Ozeanograph'),google_rank_labels.index(u'Ozeanograph')
for i in google_dif.keys():
    if wiki_bias_rank.has_key(i):
        google_rank_dict[i]=[google_rank[google_dif.keys().index(i)],wiki_bias_rank[i]]
    else:
        google_rank_dict[i]=[google_rank[google_dif.keys().index(i)],None]
print len(google_rank_labels),len(google_rank_dict)

###Prediction example

In [22]:
# Code source: Gaël Varoquaux
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

h = .02  # step size in the mesh

logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X, Y)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
from sklearn.cross_validation import train_test_split
# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, Y)
print X_train.shape, X_test.shape
logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.
logreg_fit=logreg.fit(X_train, y_train)
print logreg_fit.intercept_
y_pred = logreg_fit.predict(X_test)
print("{0} / {1} correct".format(np.sum(y_test == y_pred), len(y_test)))
print y_pred

(112, 2) (38, 2)
[ 96.61635458   9.13595029 -15.66607786]
28 / 38 correct
[1 2 0 1 0 2 0 2 0 1 1 1 0 1 2 0 2 2 0 0 1 2 2 0 0 1 2 1 0 2 2 0 2 0 1 0 0
 2]
